In [1]:
%load_ext autoreload
%autoreload 2
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import torch, copy
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
import torch.nn.functional as F
import torchvision
import  matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from module.model import TwoLayerNet
from module.Weight_tune import *
from module.Reorg import *
from module.Cram import *
from module.init import *
from module.lts import *
from module.utils import *
from module.data import *
import datetime





# New Learning mechanism

### Data

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dtype = torch.float64

data = pd.read_csv('Copper_forecasting_data.csv')

sc = StandardScaler()
X = data.drop(["y"], axis = 1)
X = sc.fit_transform(X)
y = data["y"] / 1000

train_size = int(len(X)*0.8)
batch_size = 30

X_train = X[:train_size, :]
y_train = y[:train_size]
X_test = X[train_size:, :]
y_test = y[train_size:] 

X_train = torch.tensor(np.array(X_train), dtype=dtype)
X_test = torch.tensor(np.array(X_test), dtype=dtype)
y_train = torch.tensor(np.array(y_train), dtype=dtype)
y_test = torch.tensor(np.array(y_test), dtype=dtype)
input_dim = X_train.shape[1]
    

test_loader = torch.utils.data.DataLoader(
    MyDataset(X_test.to(device), y_test.to(device)), 
    batch_size = batch_size, 
    shuffle=False, 
    drop_last = False)


### Model

In [3]:
model = TwoLayerNet(input_dim, 1, 1)

### Config

In [4]:
# learning goal: max eps < learning goal
learning_goal = torch.exp(torch.tensor(1)).to(dtype = dtype)
lr_rate = .01
lr_bound = 1e-5
criterion = nn.MSELoss()
epochs = 50
"""
# Note
1. hidden dim should check the previous model. dynamically change
2. For lr_rate, lr bound, lr goal (eps bound) are all the same fro eahc module
"""
config_wt = {
    "epochs": epochs,
    "criterion": criterion,        # loss function
    "lr_rate": lr_rate,            # learning rate 
    "lr_bound": lr_bound,          # lower bound of learning rate 
    "lr_goal": learning_goal,      # if regular eps < eps_reg: accept the model
}

config_cram = {
    "lr_goal": learning_goal, 
    "s": 0.001,                     # a small num in cram
}

config_reorg  = {
    "epochs": epochs,
    "criterion": criterion,        # loss function
    "lr_rate": lr_rate,            # learning rate 
    "lr_bound": lr_bound,          # lower bound of learning rate 
    "lr_goal": learning_goal,      # if regular eps < eps_reg: accept the model
    "print_reg": False,            # print detail, eg. loss for each epoch, or not
    "print_w_tune": False,         # print detail, eg. loss for each epoch, or not
    "validate_run": False,         # validate the model, or not
}
# NOTE 
# 1. for leaning goals, if first using weightune and no LTS or otherthings, 
# 13 for getting acceptable wt | 10 not acceptable


### Full Path

- L11 p9, third learning mechanism\
`Check: should y add .reshape(-1, 1)? since some warning shows when testing the final model`\
`Check validate section`

In [ ]:
# check the random of init. Should not be random in init.
k = 0
loss = []
while k<2:
    model = init_model(X_train, y_train)
    # 2. obtaining_LTS / selecting_LTS
    train_loader, indices, n = lts(model, X_train, y_train, learning_goal)

    # 3. check learning goal
    acceptable, model, train_loss_list, test_loss_list = \
        module_weight_EU_LG_UA(model, train_loader, test_loader, **config_wt)
    acceptable, eps, y_pred = check_acceptable(train_loader, model, learning_goal)
    loss.append(train_loss_list)
    k+=1

In [14]:
# Result log file
file_create_time = str(datetime.datetime.now().date())
out_file = open(f"result_log/{file_create_time}" + '.txt', 'a')
out_file.write(f"################################## New lts #######################################\n")

83

In [ ]:
#################################################################
# Full step for the learning algorithm mechanism
# NOTE
# 1. model right before reorg always need to be acceptable model
# 2. model after cram and reorg always need to be acceptable 
# 3. check for the above every time after cram and reorg
# 4. the randomness: in cram find r
#################################################################

# 1. initializing_1_ReLU_LR | L11 p2
out_file.write("##############################################################################")
out_file.write("##############################################################################")
out_file.write("##############################################################################")
out_file.write(str(datetime.datetime.now()))
out_file.write("##############################################################################")
out_file.write("##############################################################################")
out_file.write("##############################################################################")
model = init_model(X_train, y_train)
n = 0
n_not_fit = 5
model_history = {}


while n < len(X_train):
    out_file.write(f"### New lts ###\n")
    models_within_iter = {}
    # 2. obtaining_LTS / selecting_LTS
    train_loader, indices, X_train_lts, y_train_lts, n = \
        lts(model, X_train, y_train, learning_goal, n_not_fit, out_file)

    # 3. check learning goal
    acceptable, eps_sqaure, y_pred = check_acceptable(train_loader, model, learning_goal)
    models_within_iter["begin"] = model
    if acceptable:
        continue

    else:
        torch.save(model, "unacceptable/selecting.pth")

        # store model in acceptable/wt.pth if acceptable
        # store model in unacceptable/wt.pth if not acceptable
        out_file.write(f"//////////// Start module_EU_LG_UA Epoch ///////////\n")
        print(f"//////////// Start module_EU_LG_UA Epoch ///////////")
        acceptable, model, train_loss_list, test_loss_list = \
            module_weight_EU_LG_UA(model, train_loader, test_loader, out_file, **config_wt)
        models_within_iter["wt"] = model
        
        if acceptable:
            out_file.write("////////// Start REORG with accpetable wt //////////\n")   
            out_file.write(f"model after wt: {model}\n")
            print("////////// Start REORG with accpetable wt //////////")   
            print(f"model after wt: {model}")
            
            # load model in acceptable/wt.pth if wt acceptable
            pre_module = "wt"
            reorg = reorganising(pre_module, train_loader, test_loader, out_file, **config_reorg)
            reorg.reorganising()
            model = reorg.model
            models_within_iter["reorg"] = model
            out_file.write(f"model after reorg: {model}\n")
            print(f"model after reorg: {model}")

        else:
            out_file.write("////////// Start CRAM and REORG with unacceptable wt //////////\n")
            print("////////// Start CRAM and REORG with unacceptable wt //////////")

            # load model before wt: unacceptable/selecting.pth if unacceptable after wt
            model = torch.load("unacceptable/selecting.pth")    
            acceptable, eps_square, y_pred = check_acceptable(train_loader, model, learning_goal)
            out_file.write(f"model after wt: {model}\n")
            out_file.write(f"eps_square (last 10) before cram: {eps_square[-10:]}\n")
            print(f"model after wt: {model}")
            print(f"eps_square (last 10) before cram: {eps_square[-10:]}")

            # load model in unacceptable/wt.pth if wt not acceptable       
            # store acceptable cram in acceptable/cram.pth 
            cram = cramming(model, X_train[indices], y_train[indices], out_file, **config_cram)            
            cram.cram() 
            model = cram.model
            models_within_iter["cram"] = model
            out_file.write(f"model after cram: {model}\n")  
            print(f"model after cram: {model}")      
            # check accetable
            acceptable, eps_square, y_pred = check_acceptable(train_loader, model, learning_goal)
            out_file.write(f"eps_sqaure (last 10) after cram: {eps_square[-10:]}\n")
            print(f"eps_sqaure (last 10) after cram: {eps_square[-10:]}")
            if not acceptable:
                out_file.write("weird cram\n")
                print("weird cram")
                break

            # load model in acceptable/cram.pth if cram acceptable
            # store acceptable cram in acceptable/cram.pth
            pre_module = "cram"
            reorg = reorganising(pre_module, train_loader, test_loader, out_file, **config_reorg)
            reorg.reorganising()
            model = reorg.model
            models_within_iter["reorg"] = model
            out_file.write(f"model after reorg: {model}\n")
            print(f"model after reorg: {model}")

            # check 
            acceptable, eps_square, y_pred = check_acceptable(train_loader, model, learning_goal)
            if acceptable:
                pass
            else:
                out_file.write("weird reorg\n")
                print("weird reorg")
                break
            
    model_history[n] = models_within_iter

    

In [16]:
# final model file
import pickle
with open("result/updated_models.pk", "wb") as f:
    pickle.dump(model_history, f)
out_file.close()

with open("result/updated_models.pk", "rb") as f:
    model_history = pickle.load(f)

In [20]:
# all train loader
triain_loader = torch.utils.data.DataLoader(
    MyDataset(X_train.to(device), y_train.to(device)), 
    batch_size = batch_size, 
    shuffle=False, 
    drop_last = False)


In [40]:
# validate
train_loss = 0
for x, y in train_loader:
    output = model(x)
    loss = criterion(output, y.reshape(-1, 1))
    train_loss += loss.item()
train_loss/=len(train_loader)

test_loss = 0
for x, y in test_loader:
    output = model(x)
    loss = criterion(output, y.reshape(-1, 1))
    test_loss += loss.item()
test_loss/=len(test_loader)

train_loss, test_loss

(1.5053799064934228, 19.585845570990124)

### Benchmark

In [39]:
# two layers
bench_model = TwoLayerNet(18, 50, 1)
criterion = nn.MSELoss()
optimizer = optim.SGD(bench_model.parameters(), lr=0.001)

for i in range(50):
    for x, y in train_loader:
        preds = bench_model(x)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    
train_loss = 0
for x, y in train_loader:
    output = bench_model(x).reshape(-1, 1)
    loss = criterion(output, y.reshape(-1, 1))
    train_loss += loss.item()
train_loss/=len(train_loader)

test_loss = 0
for x, y in test_loader:
    output = bench_model(x).reshape(-1, 1)
    loss = criterion(output, y.reshape(-1, 1))
    test_loss += loss.item()
test_loss/=len(test_loader)

train_loss, test_loss

(6.835551759396442, 887.2577571347302)